In [1]:
import requests
import pandas as pd
import itertools

데이터 전처리란 획득한 데이터를 필요한 형태로 가공하고 정제하는 작업을 의미합니다. 우리가 수집한 데이터는 몇 가지 문제가 있습니다. 이러한 문제를 해결하기 위해 데이터 전처리를 진행하겠습니다.

1. 동명이인인 국회의원이 존재합니다.
2. 필요 없는 데이터 칼럼이 다수 존재합니다.
3. 네트워크 분석을 위해서는 국회의원을 node 리스트로 만들어야 합니다.
4. 네트워크 분석을 위해서는 공동 발의에 참여한 국회의원들 간의 관계를 나타내는 edge 리스트가 필요합니다.


## 1. 동명이인 국회의원 확인 및 처리

- 제21대: 더불어민주당 김병욱 의원과 미래통합당 김병욱 의원, 더불어민주당 이수진 의원과 더불어시민당(추후 합당됨) 이수진 의원
  - 더불어민주당 김병욱 의원: 김병욱A
  - 미래통합당 김병욱 의원: 김병욱B
  - 더불어민주당 이수진 의원: 이수진A
  - 더불어시민당 이수진 의원: 이수진B
- 제20대: 새누리당 최경환 의원과 국민의당 최경환 의원, 새누리당 김성태 의원과 새누리당 (비례) 김성태 의원
  - 새누리당 최경환 의원: 최경환A
  - 국민의당 최경환 의원: 최경환B
  - 새누리당 김성태 의원: 김성태A
  - 새누리당 (비례) 김성태 의원: 김성태B


In [36]:
# 데이터 불러오기
lawmaker_20 = pd.read_csv("./data/lawmaker_20.csv")
lawmaker_21 = pd.read_csv("./data/lawmaker_21.csv")
laws_20 = pd.read_csv("./data/laws_20.csv")
laws_21 = pd.read_csv("./data/laws_21.csv")

In [13]:
laws_20["PUBL_PROPOSER"]

,BILL_ID,BILL_NO,BILL_NAME,COMMITTEE,PROPOSE_DT,PROC_RESULT,AGE,DETAIL_LINK,PROPOSER,MEMBER_LIST,...,LAW_SUBMIT_DT,CMT_PROC_RESULT_CD,CMT_PROC_DT,CMT_PRESENT_DT,COMMITTEE_DT,PROC_DT,COMMITTEE_ID,PUBL_PROPOSER,LAW_PROC_RESULT_CD,RST_PROPOSER
0,PRC_K2F0C0Y5B2D2C1Z5B2F6T4S2K9N8V8,2024996,집합건물의 소유 및 관리에 관한 법률 일부개정법률안,법제사법위원회,2020-05-22,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,김병관의원 등 13인,http://likms.assembly.go.kr/bill/coactorListPo...,...,NaN,NaN,NaN,NaN,2020-05-25,2020-05-29,9700006.0,"고용진,김병욱,김철민,박재호,박정,서형수,심기준,위성곤,유승희,이규희,이원욱,홍익표",NaN,김병관
1,PRC_R2A0X0T5I2I2R1I4B5A6P3G2S7W8P0,2024995,지방세법 일부개정법률안,행정안전위원회,2020-05-22,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,김병관의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,...,NaN,NaN,NaN,NaN,2020-05-25,2020-05-29,9700480.0,"김철민,박재호,서형수,심기준,위성곤,유승희,이규희,이원욱,이종걸",NaN,김병관
2,PRC_Z2T0H0A5G1H8J1M0Q4W4S0F4E6G6Y4,2024933,상장회사법안,정무위원회,2020-05-18,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,채이배의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,...,NaN,NaN,NaN,NaN,2020-05-19,2020-05-29,9700008.0,"김관영,김동철,김종훈,박선숙,박찬대,이용득,이철희,정인화,추혜선",NaN,채이배
3,PRC_P2J0Q0D5N1R2O1L7Y1E2M0D1S8T1I9,2024932,자원의 절약과 재활용촉진에 관한 법률 일부개정법률안,환경노동위원회,2020-05-12,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,문진국의원 등 11인,http://likms.assembly.go.kr/bill/coactorListPo...,...,NaN,NaN,NaN,NaN,2020-05-13,2020-05-29,9700038.0,"강석호,강효상,김규환,김명연,김석기,김용태,민경욱,이철규,정갑윤,정운천",NaN,문진국
4,PRC_S2F0R0V5W1L1H1Q8K3D1G2L1D3V9M6,2024931,건설기술 진흥법 일부개정법률안,국토교통위원회,2020-05-11,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,송석준의원 등 11인,http://likms.assembly.go.kr/bill/coactorListPo...,...,NaN,NaN,NaN,NaN,2020-05-12,2020-05-29,9700407.0,"김석기,김정재,성일종,송언석,송희경,유동수,윤종필,이양수,이종배,함진규",NaN,송석준
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21589,PRC_S1J6G0G5Q3A0J0H9A1F5Z1T4V8H7L6,2000005,교육기본법 일부개정법률안,교육위원회,2016-05-30,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,이찬열의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,...,NaN,NaN,NaN,2016-11-16,2016-06-13,2020-05-29,9700512.0,"김경협,백재현,변재일,안규백,오제세,원혜영,이춘석,정성호,조정식",NaN,이찬열
21590,PRC_M1K6X0P5B3E0V0X9T3U4I3P0A4E3M6,2000004,정부조직법 일부개정법률안,행정안전위원회,2016-05-30,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,이종배의원 등 11인,http://likms.assembly.go.kr/bill/coactorListPo...,...,NaN,NaN,NaN,2016-11-07,2016-06-13,2020-05-29,9700480.0,"권성동,김재경,박덕흠,박명재,배덕광,안상수,이철우,이학재,이현재,주호영",NaN,이종배
21591,PRC_W1V6P0D5G3U0I0E9Q2Q1G3C2W7G7I7,2000003,근로기준법 일부개정법률안,환경노동위원회,2016-05-30,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,이찬열의원 등 11인,http://likms.assembly.go.kr/bill/coactorListPo...,...,NaN,NaN,NaN,2016-11-21,2016-06-13,2020-05-29,9700038.0,"김경협,김현미,민병두,박홍근,변재일,안규백,오제세,원혜영,이춘석,정성호",NaN,이찬열
21592,PRC_E1Z6M0V5E3V0A0G9C0G9K2M6Q5Y5S8,2000002,빅데이터의 이용 및 산업진흥 등에 관한 법률안,과학기술정보방송통신위원회,2016-05-30,임기만료폐기,20,http://likms.assembly.go.kr/bill/billDetail.do...,배덕광의원 등 14인,http://likms.assembly.go.kr/bill/coactorListPo...,...,NaN,폐기,2020-03-05,2016-11-09,2016-06-13,2020-05-29,9700479.0,"김세연,김정훈,김한표,박맹우,박주선,유기준,유재중,이종배,이진복,이채익,이헌승,조경...",NaN,배덕광


In [ ]:
def disambiguate20(x):
    if pd.isnull(x["PUBL_PROPOSER"]):
        proposers = []
    else:
        proposers = x["PUBL_PROPOSER"].split(",")
    repr_proposer = x["RST_PROPOSER"]
    if "," in repr_proposer:
        repr_proposer = repr_proposer.split(",")
    else:
        repr_proposer = [repr_proposer]
    proposers_list = x["MEMBER_LIST"]
    proposers.extend(repr_proposer)

    if "최경환" not in proposers and "김성태" not in proposers:
        x["proposers"] = proposers
        x["repr_proposer"] = repr_proposer
        return x
    else:
        # 최경환이 있는 경우
        if "최경환" in proposers:
            page = requests.get(proposers_list).text
            a = "崔炅煥"  # 새누리당 최경환
            b = "崔敬煥"  # 국민의당 최경환
            if a in page:
                proposers[proposers.index("최경환")] = "최경환A"
            elif b in page:
                proposers[proposers.index("최경환")] = "최경환B"
        # 김성태가 있는 경우
        if "김성태" in proposers:
            page = requests.get(proposers_list).text
            a = "金聖泰"  # 새누리당 김성태
            b = "金成泰"  # 새누리당 (비례) 김성태
            if a in page:
                proposers[proposers.index("김성태")] = "김성태A"
            elif b in page:
                proposers[proposers.index("김성태")] = "김성태B"
        x["proposers"] = proposers
        # 대표발의자가 최경환이나 김성태인 경우
        if "A" in proposers[-1] or "B" in proposers[-1]:
            x["repr_proposer"] = [proposers[-1]]
        else:
            x["repr_proposer"] = repr_proposer
        return x


laws_20 = laws_20.apply(disambiguate20, axis="columns")

In [92]:
# lawmaker_20 데이터프레임에서 이름 변경
lawmaker_20.loc[lawmaker_20[lawmaker_20["HJ_NM"] == "崔炅煥"].index, "HG_NM"] = (
    "최경환A"
)
lawmaker_20.loc[lawmaker_20[lawmaker_20["HJ_NM"] == "崔敬煥"].index, "HG_NM"] = (
    "최경환B"
)
lawmaker_20.loc[lawmaker_20[lawmaker_20["HJ_NM"] == "金聖泰"].index, "HG_NM"] = (
    "김성태A"
)
lawmaker_20.loc[lawmaker_20[lawmaker_20["HJ_NM"] == "金成泰"].index, "HG_NM"] = (
    "김성태B"
)

In [118]:
def disambiguate21(x):
    if pd.isnull(x["PUBL_PROPOSER"]):
        proposers = []
    else:
        proposers = x["PUBL_PROPOSER"].split(",")
    repr_proposer = x["RST_PROPOSER"]
    if "," in repr_proposer:
        repr_proposer = repr_proposer.split(",")
    else:
        repr_proposer = [repr_proposer]
    proposers_list = x["MEMBER_LIST"]
    proposers.extend(repr_proposer)

    if "김병욱" not in proposers and "이수진" not in proposers:
        x["proposers"] = proposers
        x["repr_proposer"] = repr_proposer
        return x
    else:
        # 김병욱이 있는 경우. 둘은 한자 까지 같다...
        if "김병욱" in proposers:
            page = requests.get(proposers_list).text
            a = "김병욱(더불어민주당/金炳旭)"  # 더불어민주당 김병욱
            if a in page:
                proposers[proposers.index("김병욱")] = "김병욱A"
            else:  # 국민의힘은 정당명이 자주 바뀌었기 때문에 else 처리한다.
                proposers[proposers.index("김병욱")] = "김병욱B"
        # 이수진이 있는 경우
        if "이수진" in proposers:
            page = requests.get(proposers_list).text
            a = "李秀眞"  # 더불어민주당 이수진
            b = "李壽珍"  # 더불어시민당 이수진
            if a in page:
                proposers[proposers.index("이수진")] = "이수진A"
            elif b in page:
                proposers[proposers.index("이수진")] = "이수진B"
        x["proposers"] = proposers
        # 대표발의자가 김병욱이나 이수진인 경우
        if "A" in proposers[-1] or "B" in proposers[-1]:
            x["repr_proposer"] = [proposers[-1]]
        else:
            x["repr_proposer"] = repr_proposer
        return x


laws_21 = laws_21.apply(disambiguate21, axis="columns")

,HG_NM,HJ_NM,ENG_NM,BTH_GBN_NM,BTH_DATE,JOB_RES_NM,POLY_NM,ORIG_NM,ELECT_GBN_NM,CMIT_NM,...,HOMEPAGE,STAFF,SECRETARY,SECRETARY2,MONA_CD,MEM_TITLE,ASSEM_ADDR,UNIT_CD,UNIT_NM,elected_cnt
34,김병욱,金炳旭,KIM BYONGWOOK,양,1977-04-30,위원,국민의미래,경북 포항시남구울릉군,지역구,과학기술정보방송통신위원회,...,https://blog.naver.com/ilungkim,"김민정, 남호균","오세웅, 이동찬","박민지, 이성현, 김대민, 윤영하, 김복순",KB04377U,김병욱 국회의원 (포항시남구울릉군)\r\n\r\n포항고 졸업\r\n경북대 정치외교학...,의원회관 320호,NaN,NaN,1
35,김병욱,金炳旭,KIM BYUNGWOOK,음,1965-04-15,위원,더불어민주당,경기 성남시분당구을,지역구,"국토교통위원회, 예산결산특별위원회",...,http://blog.naver.com/777byung,"왕홍곤, 최현","박채운, 조제윤","김정은, 이종목, 김경은, 박상은, 곽정훈",GFF1986K,[더불어민주당 김병욱 국회의원(분당을)]\r\n\r\n부산 배정고 졸업\r\n한양대...,의원회관 510호,NaN,NaN,2


,HG_NM,HJ_NM,ENG_NM,BTH_GBN_NM,BTH_DATE,JOB_RES_NM,POLY_NM,ORIG_NM,ELECT_GBN_NM,CMIT_NM,...,HOMEPAGE,STAFF,SECRETARY,SECRETARY2,MONA_CD,MEM_TITLE,ASSEM_ADDR,UNIT_CD,UNIT_NM,elected_cnt
35,김병욱,金炳旭,KIM BYUNGWOOK,음,1965-04-15,위원,더불어민주당,경기 성남시분당구을,지역구,"국토교통위원회, 예산결산특별위원회",...,http://blog.naver.com/777byung,"왕홍곤, 최현","박채운, 조제윤","김정은, 이종목, 김경은, 박상은, 곽정훈",GFF1986K,[더불어민주당 김병욱 국회의원(분당을)]\r\n\r\n부산 배정고 졸업\r\n한양대...,의원회관 510호,NaN,NaN,2


In [135]:
# lawmaker_21 데이터프레임에서 이름 변경
lawmaker_21.loc[lawmaker_21[lawmaker_21["HJ_NM"] == "李秀眞"].index, "HG_NM"] = (
    "이수진A"
)
lawmaker_21.loc[lawmaker_21[lawmaker_21["HJ_NM"] == "李壽珍"].index, "HG_NM"] = (
    "이수진B"
)
lawmaker_21.loc[
    lawmaker_21[
        (lawmaker_21["HG_NM"] == "김병욱") & (lawmaker_21["POLY_NM"] == "더불어민주당")
    ].index,
    "HG_NM",
] = "김병욱A"
lawmaker_21.loc[
    lawmaker_21[
        (lawmaker_21["HG_NM"] == "김병욱") & (lawmaker_21["POLY_NM"] == "더불어민주당")
    ].index,
    "HG_NM",
] = "김병욱B"

## 2. 필요 없는 칼럼 제거 및 데이터 정제


In [82]:
laws_20.columns

Index(['BILL_ID', 'BILL_NO', 'BILL_NAME', 'COMMITTEE', 'PROPOSE_DT',
       'PROC_RESULT', 'AGE', 'DETAIL_LINK', 'PROPOSER', 'MEMBER_LIST',
       'LAW_PROC_DT', 'LAW_PRESENT_DT', 'LAW_SUBMIT_DT', 'CMT_PROC_RESULT_CD',
       'CMT_PROC_DT', 'CMT_PRESENT_DT', 'COMMITTEE_DT', 'PROC_DT',
       'COMMITTEE_ID', 'PUBL_PROPOSER', 'LAW_PROC_RESULT_CD', 'RST_PROPOSER',
       'proposers', 'repr_proposer'],
      dtype='object')

In [112]:
# 20대 법률 발의안 데이터에서 필요한 칼럼만 추출하여 저장한다.
laws_20_preprocessed = laws_20[
    ["BILL_ID", "BILL_NAME", "MEMBER_LIST", "proposers", "repr_proposer"]
]
laws_20_preprocessed.to_csv("./data/laws_20_preprocessed.csv", index=False)

In [136]:
# 21대도 마찬가지
laws_21_preprocessed = laws_21[
    ["BILL_ID", "BILL_NAME", "MEMBER_LIST", "proposers", "repr_proposer"]
]
laws_21_preprocessed.to_csv("./data/laws_21_preprocessed.csv", index=False)

In [143]:
# lawmaker 테이블도 필요한 칼럼만 추출하여 저장한다.
lawmaker_20["elected_cnt"] = lawmaker_20["UNITS"].str.split(",").apply(len)
lawmaker_20_preprocessed = lawmaker_20[
    [
        "HG_NM",
        "HG_NM",
        "HJ_NM",
        "BTH_DATE",
        "SEX_GBN_NM",
        "elected_cnt",
        "POLY_NM",
        "ORIG_NM",
    ]
]
lawmaker_21["elected_cnt"] = lawmaker_21["UNITS"].str.split(",").apply(len)
lawmaker_21_preprocessed = lawmaker_21[
    [
        "HG_NM",
        "HG_NM",
        "HJ_NM",
        "BTH_DATE",
        "SEX_GBN_NM",
        "elected_cnt",
        "POLY_NM",
        "ORIG_NM",
    ]
]
lawmaker_20_preprocessed.to_csv("./data/lawmaker_20_preprocessed.csv", index=False)
lawmaker_21_preprocessed.to_csv("./data/lawmaker_21_preprocessed.csv", index=False)

## 3. 국회의원을 node 리스트로 만들기


In [144]:
lawmaker_20_nodes = lawmaker_20_preprocessed
lawmaker_21_nodes = lawmaker_21_preprocessed
lawmaker_20_nodes.columns = [
    "Id",
    "Label",
    "Label_Hanja",
    "Birth",
    "Gender",
    "Elected",
    "Party",
    "Region",
]
lawmaker_21_nodes.columns = [
    "Id",
    "Label",
    "Label_Hanja",
    "Birth",
    "Gender",
    "Elected",
    "Party",
    "Region",
]

In [145]:
lawmaker_20_nodes.to_csv("./data/lawmaker_20_nodes.csv", index=False)
lawmaker_21_nodes.to_csv("./data/lawmaker_21_nodes.csv", index=False)

## 4. 공동 발의에 참여한 국회의원들 간의 관계를 나타내는 edge 리스트 만들기


In [140]:
pairs_20 = (
    laws_20_preprocessed["proposers"]
    .apply(lambda x: x if x is None else list(itertools.combinations(x, 2)))
    .explode()
)
pairs_20 = pd.DataFrame(pairs_20.tolist(), columns=["Source", "Target"])
pairs_20 = pairs_20[pairs_20["Target"].notnull()]
pairs_20.to_csv("./data/lawmaker_20_edges.csv", index=False)

In [141]:
pairs_21 = (
    laws_20_preprocessed["proposers"]
    .apply(lambda x: x if x is None else list(itertools.combinations(x, 2)))
    .explode()
)
pairs_21 = pd.DataFrame(pairs_21.tolist(), columns=["Source", "Target"])
pairs_21 = pairs_21[pairs_21["Target"].notnull()]
pairs_21.to_csv("./data/lawmaker_21_edges.csv", index=False)